In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras import models
from tensorflow.compat.v1 import ConfigProto, InteractiveSession
import numpy as np
from PIL import Image
import random

config = ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = InteractiveSession(config=config)

In [2]:
cnn=tf.keras.models.load_model('mango_cnn.h5')
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 196, 196, 32)      9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 196, 196, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 94, 94, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 94, 94, 64)        2

In [3]:
label=pd.read_csv('TrainDev/train.csv',header=None)
label=label.dropna(axis=1,how='all')
data=label[[0,1,2,3,4,5]]
for i in range(1,int((label.shape[1]-1)/5)):
    current=5*i
    add_index=[0,current+1,current+2,current+3,current+4,current+5]
    add_index=label[add_index]
    add_index.columns=[0,1,2,3,4,5]
    data=data.append(add_index,ignore_index=True)
data=data.dropna(axis=0)
data.columns=['file','x1','y1','x2','y2','damage']
data['x2']+=data['x1']
data['y2']+=data['y1']
data.index=[i for i in range(data.shape[0])]
data

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,105,110,115,120,125,130,135) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,file,x1,y1,x2,y2,damage
0,38414.jpg,46.0,146.0,622.0,720.0,不良-機械傷害
1,03182.jpg,581.0,277.0,678.0,370.0,不良-機械傷害
2,29863.jpg,514.0,538.0,631.0,682.0,不良-機械傷害
3,17937.jpg,658.0,263.0,717.0,337.0,不良-機械傷害
4,40878.jpg,432.0,160.0,548.0,292.0,不良-機械傷害
...,...,...,...,...,...,...
43365,02176.jpg,838.0,636.0,868.0,657.0,不良-炭疽病
43366,48869.jpg,181.0,435.0,229.0,473.0,不良-炭疽病
43367,02176.jpg,547.0,240.0,567.0,258.0,不良-炭疽病
43368,02176.jpg,514.0,257.0,530.0,271.0,不良-炭疽病


In [14]:
x_test=[]
y_test=[]
for _ in range(1000):
    i=random.randint(data.shape[0]/2,data.shape[0]-1)
    x=Image.open('TrainDev/Pics/'+data['file'][i])
    x=x.crop((data['x1'][i],data['y1'][i],data['x2'][i],data['y2'][i]))
    x=x.resize((200,200))
    x=np.array(x)
    x_test.append(x)
    y_test.append(data['damage'][i])
x_test=np.array(x_test)
# y_test=data['damage']
# y_test=y_test[-1000:]
y_test=pd.get_dummies(y_test)
y_test=y_test.to_numpy()
print('y_test shape: ',y_test.shape)
print('x_test shape: ',x_test.shape)

y_test shape:  (1000, 5)
x_test shape:  (1000, 200, 200, 3)


In [15]:
pred=cnn.predict(x_test)

In [16]:
for i in range(pred.shape[0]):
    temp=np.zeros(5)
    temp[np.argmax(pred[i])]=1
    pred[i]=temp


In [17]:
from sklearn.metrics import accuracy_score

In [18]:
accuracy_score(y_test, pred)

0.803